# 환경설정

In [2]:
import os
import json
import datetime
import time
import re
from glob import glob
from tqdm import tqdm 

from pprint import pprint

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'NanumGothic'


# 경고메세지 끄기
import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 


now = datetime.datetime.today()
now = str(now)[:10]
now

'2024-07-23'

### ELK 접속

In [3]:
from elasticsearch import Elasticsearch

# Create the client instance
es = Elasticsearch("http://localhost:9200")

# Successful response!
es.info()

ObjectApiResponse({'name': 'elasticsearch', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BdPu_csRRH6ooZBADMrtfQ', 'version': {'number': '7.17.20', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'b26557f585b7d95c71a5549e571a6bcd2667697d', 'build_date': '2024-04-08T08:34:31.070382898Z', 'build_snapshot': False, 'lucene_version': '8.11.3', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

### Index 생성
- index: 테이블 이름
- id: 문서 고유번호
- body: 문서 내용

In [4]:
doc = {
    'user': 'no_name',
    'text': 'no info...',
    'timestamp': datetime.datetime.now(),
}
res = es.index(index="no-info", id=1, body=doc)
print(res['result'])

created


### 문서 확인
- index: 테이블 이름
- id: 문서 고유번호

In [5]:
res = es.get(index="no-info", id=1)
print(res['_source'])

{'user': 'no_name', 'text': 'no info...', 'timestamp': '2024-07-22T07:49:23.246870'}


### 문서 수정

In [6]:
doc = {
    'user': 'Steve',
    'text': 'New client...',
    'timestamp': datetime.datetime.now(),
}
res = es.update(index="no-info", id=1, doc=doc)
print(res['result'])

updated


In [7]:
res = es.get(index="no-info", id=1)
print(res['_source'])

{'user': 'Steve', 'text': 'New client...', 'timestamp': '2024-07-22T07:49:26.284927'}


### 문서 삭제

In [8]:
es.delete(index="no-info", id=1)

ObjectApiResponse({'_index': 'no-info', '_type': '_doc', '_id': '1', '_version': 3, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

### Index 삭제

In [9]:
index_name = 'no-info'
es.indices.delete(index=index_name, ignore=[400, 404])

ObjectApiResponse({'acknowledged': True})

### Index Mapping

In [7]:
index_settings = {
    "mappings": {
        "properties": {
            "rating": {"type": "float"},
            "title": {"type": "text",},
            "text": {"type": "text"},
            "images": {"type": "text"},
            "asin": {
                "type": "keyword",
                "fields": {
                    "text": {
                        "type": "text"
                    }
                }
            },
            "parent_asin": {
                "type": "keyword",
                "fields": {
                    "text": {
                        "type": "text"
                    }
                }
            },
            "user_id": {
                "type": "keyword",
                "fields": {
                    "text": {
                        "type": "text"
                    }
                }
            },
            "timestamp": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss.SSS"},
            "helpful_vote": {"type": "integer"},
            "verified_purchase": {"type": "boolean"},
            "average_rating": {"type": "float"},
            "rating_number": {"type": "integer"},
            "features": {"type": "text"},
            "description": {"type": "text"},
            "videos": {"type": "text"},
            "store": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "categories": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "details": {"type": "text"},
            "processed_text": {"type": "text"}
        }
    }
}


In [8]:
es.indices.create(
    index='amazon_reviews',
    body=index_settings,
    ignore=400,
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'amazon_reviews'})

In [9]:
es.indices.get_mapping(index='amazon_reviews')

ObjectApiResponse({'amazon_reviews': {'mappings': {'properties': {'asin': {'type': 'keyword', 'fields': {'text': {'type': 'text'}}}, 'average_rating': {'type': 'float'}, 'categories': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'description': {'type': 'text'}, 'details': {'type': 'text'}, 'features': {'type': 'text'}, 'helpful_vote': {'type': 'integer'}, 'images': {'type': 'text'}, 'parent_asin': {'type': 'keyword', 'fields': {'text': {'type': 'text'}}}, 'processed_text': {'type': 'text'}, 'rating': {'type': 'float'}, 'rating_number': {'type': 'integer'}, 'store': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'text': {'type': 'text'}, 'timestamp': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS'}, 'title': {'type': 'text'}, 'user_id': {'type': 'keyword', 'fields': {'text': {'type': 'text'}}}, 'verified_purchase': {'type': 'boolean'}, 'videos': {'type': 'text'}}}}})

### Bulk 인덱싱

In [10]:
df = pd.read_csv("amazon_reviews_merged.csv")
df = df.fillna('')
df.head()

,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,year,month,day,hour,minute,second,microsecond,review_length,main_category,title_y,average_rating,rating_number,features,description,images_y,videos,store,categories,details,processed_text
0,5.0,Great pictures and recipes,Wonderful recipes in this magazine.,[],B00HLSSQKK,B00HLSSQKK,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,2020-12-15 15:29:02.433,0,1,2020,12,15,15,29,2,433000,5,Magazine Subscriptions,Cooking With Paula Deen,4.6,538,[],[],[{'thumb': 'https://m.media-amazon.com/images/...,[],Hoffman Media Inc,"['Magazine Subscriptions', 'Cooking, Food & Wi...","{'Date First Available': 'December 30, 2013'}",wonderful recipes in this magazine
1,4.0,great for kids who love sports!,Great sports magazine that's on my 9 year olds...,[],B002PXW04Y,B002PXW04Y,AHORTSSMI6ZZFUFWMPT4UFR2ISUQ,2016-08-12 03:28:32.000,1,1,2016,8,12,3,28,32,0,10,Magazine Subscriptions,Sports Illustrated KIDS Print Magazine,4.3,2363,[],[],[{'thumb': 'https://m.media-amazon.com/images/...,[],Maven,"['Magazine Subscriptions', 'Sports, Recreation...","{'Date First Available': 'September 25, 2014'}",great sports magazine thats on my 9 year olds ...
2,5.0,A great look at what's new on the kosher scene...,"""Joy of Kosher"" magazine fills a much-needed n...",[],B00HCR5090,B00HCR5090,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,2014-04-06 23:32:53.000,4,0,2014,4,6,23,32,53,0,532,Magazine Subscriptions,Joy of Kosher With Jamie Geller Print Magazine,5.0,5,[],"['Joy of Kosher magazine is an exciting, high ...",[{'thumb': 'https://m.media-amazon.com/images/...,[],Kosher Media Network Llc,"['Magazine Subscriptions', 'Cooking, Food & Wi...","{'Date First Available': 'December 16, 2013', ...",joy of kosher magazine fills a muchneeded nich...
3,5.0,"If you enjoy reading an eye-catching, easy-to-...",I've been addicted to Martha Stewart's Everyda...,[],B0000ARXXS,B0000ARXXS,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,2007-06-15 04:04:15.000,4,0,2007,6,15,4,4,15,0,324,Magazine Subscriptions,Everyday Food,3.6,52,[],[],[{'thumb': 'https://m.media-amazon.com/images/...,[],,[],"{'Date First Available': 'April 30, 2015'}",ive been addicted to martha stewarts everyday ...
4,1.0,Too many “ gear ads” !!,Too many ads!,[],B0025ZOVEO,B0025ZOVEO,AFUB7CHTXRPD447QVQCHBZVN2IPQ,2020-06-01 21:24:40.246,13,1,2020,6,1,21,24,40,246000,3,Magazine Subscriptions,Outside Print Magazine,4.3,355,[],"['Product Description', ""Outside covers the ex...",[{'thumb': 'https://m.media-amazon.com/images/...,[],Outside,"['Magazine Subscriptions', 'Sports, Recreation...","{'Date First Available': 'April 8, 2009', 'Man...",too many ads


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71497 entries, 0 to 71496
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             71497 non-null  float64
 1   title_x            71497 non-null  object 
 2   text               71497 non-null  object 
 3   images_x           71497 non-null  object 
 4   asin               71497 non-null  object 
 5   parent_asin        71497 non-null  object 
 6   user_id            71497 non-null  object 
 7   timestamp          71497 non-null  object 
 8   helpful_vote       71497 non-null  int64  
 9   verified_purchase  71497 non-null  int64  
 10  year               71497 non-null  int64  
 11  month              71497 non-null  int64  
 12  day                71497 non-null  int64  
 13  hour               71497 non-null  int64  
 14  minute             71497 non-null  int64  
 15  second             71497 non-null  int64  
 16  microsecond        714

In [12]:
from elasticsearch.helpers import bulk
from tqdm import tqdm

action_list = []
for idx, row in tqdm(df.iterrows()):

    doc = {
            "rating": row["rating"],
            "title": row["title_x"],
            "text": row["text"],
            "images": row["images_x"],
            "asin": row["asin"],
            "parent_asin": row["parent_asin"],
            "user_id": row["user_id"],
            "timestamp": row["timestamp"],
            "helpful_vote": row["helpful_vote"],
            "verified_purchase": bool(row["verified_purchase"]),
            "average_rating": row["average_rating"],
            "rating_number": row["rating_number"],
            "features": row["features"],
            "description": row["description"],
            "videos": row["videos"],
            "store": row["store"],
            "categories": row["categories"],
            "details": row["details"],
            "processed_text": row["processed_text"],
    }
        
        
    record ={
        '_index': 'amazon_reviews',
        '_source': doc,
    }

    action_list.append(record)

bulk(es, action_list, raise_on_error=False)

71497it [00:07, 9785.40it/s] 


(71497, [])

### 검색 쿼리

### 1. 단순 키워드 검색 (Match Query)
- 특정 텍스트가 포함된 문서를 검색

In [13]:
query = {
    "query": {
        "match": {
            "categories": "Transportation"
        }
    }
}

response = es.search(index="amazon_reviews", body=query)
print(response)

{'took': 2554, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 353, 'relation': 'eq'}, 'max_score': 4.9848666, 'hits': [{'_index': 'amazon_reviews', '_type': '_doc', '_id': 'V7aX3JABKR8jdWkMocFx', '_score': 4.9848666, '_source': {'rating': 1.0, 'title': 'Nothing like it used to be.', 'text': "European Car is a magazine full of eye candy.  Lots of German and Italian uber-exotics grace its pages.  While that's great, and it's enjoyable to read about such cars, EC has lost many subscribers due to its loss of focus on the more attainable European cars out there.  The pages used to be full of VW, BMW, Audi and Porsche project cars that were all within reach of your average Joe with a taste for something other than the basic Camaro or Mustang.  Many used Euro cars are very cheap and easy to get into for those that would otherwise not be able to buy a new car of the same make. I'd hoped that the focus on these cars would bec

In [14]:
pprint(response['hits'])

{'hits': [{'_id': 'V7aX3JABKR8jdWkMocFx',
           '_index': 'amazon_reviews',
           '_score': 4.9848666,
           '_source': {'asin': 'B00007AX2M',
                       'average_rating': 3.5,
                       'categories': "['Magazine Subscriptions', 'Professional "
                                     "& Educational Journals', 'Professional & "
                                     "Trade', 'Transportation']",
                       'description': '[]',
                       'details': '{}',
                       'features': '[]',
                       'helpful_vote': 15,
                       'images': '[]',
                       'parent_asin': 'B00007AX2M',
                       'processed_text': 'european car is a magazine full of '
                                         'eye candy  lots of german and '
                                         'italian uberexotics grace its pages  '
                                         'while thats great and its enjoyab

In [15]:
print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"])

Got 353 Hits:
{'rating': 1.0, 'title': 'Nothing like it used to be.', 'text': "European Car is a magazine full of eye candy.  Lots of German and Italian uber-exotics grace its pages.  While that's great, and it's enjoyable to read about such cars, EC has lost many subscribers due to its loss of focus on the more attainable European cars out there.  The pages used to be full of VW, BMW, Audi and Porsche project cars that were all within reach of your average Joe with a taste for something other than the basic Camaro or Mustang.  Many used Euro cars are very cheap and easy to get into for those that would otherwise not be able to buy a new car of the same make. I'd hoped that the focus on these cars would become more intense when Les Bidrawn took over as editor a few years back.  Unfortunately, it appeared to me that that's when things really seemed to go downhill.  I let my subscription lapse and can't see picking it back up unless the mag undergoes a major transformation.  So if you're

In [81]:
# size : 가져올 문서의 개수
response = es.search(index="amazon_reviews", size=100, body=query)

print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"])

Got 353 Hits:
{'rating': 1.0, 'title': 'Nothing like it used to be.', 'text': "European Car is a magazine full of eye candy.  Lots of German and Italian uber-exotics grace its pages.  While that's great, and it's enjoyable to read about such cars, EC has lost many subscribers due to its loss of focus on the more attainable European cars out there.  The pages used to be full of VW, BMW, Audi and Porsche project cars that were all within reach of your average Joe with a taste for something other than the basic Camaro or Mustang.  Many used Euro cars are very cheap and easy to get into for those that would otherwise not be able to buy a new car of the same make. I'd hoped that the focus on these cars would become more intense when Les Bidrawn took over as editor a few years back.  Unfortunately, it appeared to me that that's when things really seemed to go downhill.  I let my subscription lapse and can't see picking it back up unless the mag undergoes a major transformation.  So if you're

In [16]:
def fetch_all_hits_with_scroll(es, index_name, query, scroll='2m', size=1000):
    """
    Elasticsearch에서 스크롤 API를 사용하여 모든 결과를 가져오는 함수.
    
    :param es: Elasticsearch 클라이언트 객체
    :param index_name: 검색할 인덱스 이름
    :param query: Elasticsearch 쿼리
    :param scroll: 스크롤 유지 시간 (기본값 '2m')
    :param size: 한 번에 가져올 문서 수 (기본값 1000)
    :return: 모든 검색 결과 리스트
    """
    all_hits = []

    # 초기 검색 요청
    response = es.search(index=index_name, body={**query, "size": size}, scroll=scroll)
    all_hits.extend(response['hits']['hits'])
    scroll_id = response['_scroll_id']

    while True:
        # 스크롤을 사용하여 다음 결과 요청
        response = es.scroll(scroll_id=scroll_id, scroll=scroll)
        hits = response['hits']['hits']

        if not hits:
            break

        all_hits.extend(hits)
        scroll_id = response['_scroll_id']

    # 스크롤 컨텍스트 정리
    es.clear_scroll(scroll_id=scroll_id)

    return all_hits


# 모든 검색 결과 가져오기
all_hits = fetch_all_hits_with_scroll(es, index_name, query)

for hit in all_hits:
    print(hit["_source"])

{'rating': 1.0, 'title': 'Nothing like it used to be.', 'text': "European Car is a magazine full of eye candy.  Lots of German and Italian uber-exotics grace its pages.  While that's great, and it's enjoyable to read about such cars, EC has lost many subscribers due to its loss of focus on the more attainable European cars out there.  The pages used to be full of VW, BMW, Audi and Porsche project cars that were all within reach of your average Joe with a taste for something other than the basic Camaro or Mustang.  Many used Euro cars are very cheap and easy to get into for those that would otherwise not be able to buy a new car of the same make. I'd hoped that the focus on these cars would become more intense when Les Bidrawn took over as editor a few years back.  Unfortunately, it appeared to me that that's when things really seemed to go downhill.  I let my subscription lapse and can't see picking it back up unless the mag undergoes a major transformation.  So if you're into top doll

In [17]:
# 데이터프레임 변환
df_hits = pd.DataFrame(all_hits)
print(df_hits.shape)
df_hits.head()

(353, 5)


,_index,_type,_id,_score,_source
0,amazon_reviews,_doc,V7aX3JABKR8jdWkMocFx,4.984867,"{'rating': 1.0, 'title': 'Nothing like it used..."
1,amazon_reviews,_doc,k7aX3JABKR8jdWkMo8gc,4.984867,"{'rating': 5.0, 'title': 'One of my favorites�..."
2,amazon_reviews,_doc,fraX3JABKR8jdWkMo8q_,4.984867,"{'rating': 5.0, 'title': 'Great Magazine', 'te..."
3,amazon_reviews,_doc,6baX3JABKR8jdWkMo8q_,4.984867,"{'rating': 3.0, 'title': 'Three Stars', 'text'..."
4,amazon_reviews,_doc,-baX3JABKR8jdWkMpMxs,4.984867,"{'rating': 2.0, 'title': 'Heavy coverage on pi..."


### 2. 다중 매치 쿼리 (Multi-Match Query)
- 여러 필드에서 특정 텍스트를 검색

In [18]:
response = es.search(
    index="amazon_reviews",
    body={
        "query": {
            "multi_match": {
                "query": "great magazine",
                "fields": ["title", "text"]
            }
        }
    }
)

print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"])


Got 10000 Hits:
{'rating': 5.0, 'title': 'Great, Great Magazine', 'text': "To take a different perspective at first, I'm a Brit living in the States, and this is hands down the best way to keep in touch with what's happening on the worldwide music scene. I've discovered so much new music through this magazine, which is sometimes hard to find in the States. The reviews are well-written, to the point, and accurate, even taking into account differing tastes in music. Notable albums are given much more copy and make for an in-depth, and fascinating read. The rest of the mag is great too, with news, really in-depth interviews that are not afraid to ask the hard questions. Finally, cash for questions is a hoot, where readers ask their own questions and don't always take things too seriously. From memory, I think it was Elvis Costello who was asked &#34;Have you ever written a song to get a girl to sleep with you?&#34;<br />Essential reading for any music fan. I could never be without this ma

### 3. 필터 쿼리 (Term Query)
- 정확히 일치하는 값을 갖는 문서를 검색 (예: 특정 사용자의 리뷰를 검색)

In [19]:
response = es.search(
    index="amazon_reviews",
    body={
        "query": {
            "term": {
                "user_id": "AFUB7CHTXRPD447QVQCHBZVN2IPQ"
            }
        }
    }
)

print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"]["user_id"])

Got 1 Hits:
AFUB7CHTXRPD447QVQCHBZVN2IPQ


### 4. 범위 쿼리 (Range Query)
- 특정 범위 내의 값을 갖는 문서를 검색 (예: 특정 기간 내에 작성된 리뷰를 검색)

In [20]:
response = es.search(
    index="amazon_reviews",
    body={
        "query": {
            "range": {
                "timestamp": {
                    "gte": "2020-01-01",
                    "lte": "2020-01-03",
                    "format": "yyyy-MM-dd",
                }
            }
        }
    }
)

print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"]["timestamp"])

Got 132 Hits:
2020-01-02 18:26:08.762
2020-01-02 00:54:57.780
2020-01-01 23:48:40.170
2020-01-02 05:49:24.677
2020-01-02 02:13:28.282
2020-01-01 23:19:30.756
2020-01-03 11:15:48.995
2020-01-01 03:51:34.861
2020-01-03 01:25:56.100
2020-01-03 17:14:26.209


### 5. 복합 쿼리 (Bool Query)
여러 조건을 조합하여 검색. 예를 들어, 특정 기간 내에 작성된 5점짜리 리뷰를 검색

In [21]:
response = es.search(
    index="amazon_reviews",
    body={
        "query": {
            "bool": {
                "must": [
                    {"term": {"rating": 5}},
                    {
                        "range": {
                            "timestamp": {
                                "gte": "2020-01-01",
                                "lte": "2020-01-31",
                                "format": "yyyy-MM-dd",
                            }
                        }
                    }
                ]
            }
        }
    }
)

print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"]["timestamp"], hit["_source"]["rating"])


Got 616 Hits:
2020-01-25 01:00:46.114 5.0
2020-01-02 18:26:08.762 5.0
2020-01-13 20:44:16.316 5.0
2020-01-05 20:00:40.496 5.0
2020-01-23 23:49:41.982 5.0
2020-01-17 07:27:12.046 5.0
2020-01-02 00:54:57.780 5.0
2020-01-18 19:05:37.850 5.0
2020-01-06 23:16:41.528 5.0
2020-01-04 23:53:31.173 5.0


### 6. 집계 (Terms Aggregation)
- 특정 필드의 데이터를 집계. 예를 들어, 각 등급별 리뷰 수를 집계.

In [22]:
response = es.search(
    index="amazon_reviews",
    body={
        "aggs": {
            "ratings_count": {
                "terms": {
                    "field": "rating"
                }
            }
        },
        "size": 0  # 집계 결과만 반환, 문서 반환 안함
    }
)

print("Got %d Hits:" % response['hits']['total']['value'])
pprint(response['aggregations'])

Got 10000 Hits:
{'ratings_count': {'buckets': [{'doc_count': 44620, 'key': 5.0},
                               {'doc_count': 9685, 'key': 1.0},
                               {'doc_count': 8206, 'key': 4.0},
                               {'doc_count': 5033, 'key': 3.0},
                               {'doc_count': 3953, 'key': 2.0}],
                   'doc_count_error_upper_bound': 0,
                   'sum_other_doc_count': 0}}


### 7. 하이라이트 쿼리 (Highlight Query)
- 검색된 텍스트 내의 키워드를 강조하여 표시

In [23]:
response = es.search(
    index="amazon_reviews",
    body={
        "query": {
            "term": {
                "text": "magazine"
            }
        },
        "highlight": {
            "fields": {
                "text": {}
            }
        }
    }
)

print("Got %d Hits:" % response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit["_source"])
    print(hit['highlight'])

Got 10000 Hits:
{'rating': 5.0, 'title': 'Five Stars', 'text': 'The magazine is the best food magazine', 'images': '[]', 'asin': 'B001THPA58', 'parent_asin': 'B001THPA58', 'user_id': 'AFGZNSPIZSAUXINDCQYKB3ISEHQA', 'timestamp': '2018-04-02 02:30:25.043', 'helpful_vote': 0, 'verified_purchase': True, 'average_rating': 4.6, 'rating_number': 13511, 'features': '[]', 'description': '[]', 'videos': '[]', 'store': 'Hearst Magazines', 'categories': "['Magazine Subscriptions', 'Cooking, Food & Wine', 'Recipes & Techniques']", 'details': '{}', 'processed_text': 'the magazine is the best food magazine'}
{'text': ['The <em>magazine</em> is the best food <em>magazine</em>']}
{'rating': 4.0, 'title': 'Four Stars', 'text': 'Great magazine. Better than our own country magazine', 'images': '[]', 'asin': 'B000UHI334', 'parent_asin': 'B000UHI334', 'user_id': 'AFK4NKFJEKVY73TVAZJXYJCI7Z3Q', 'timestamp': '2018-05-16 20:10:49.487', 'helpful_vote': 1, 'verified_purchase': True, 'average_rating': 4.5, 'ratin

In [24]:
import pandas as pd
from IPython.display import display, HTML

# 하이라이트 결과를 저장할 리스트
highlighted_results = []

# 검색 결과에서 하이라이트 부분을 추출
for hit in response['hits']['hits']:
    source = hit['_source']
    highlight = hit['highlight'] if 'highlight' in hit else {}
    highlighted_text = highlight.get('text', [source['text']])[0]
    
    highlighted_results.append({
        'user_id': source['user_id'],
        'rating': source['rating'],
        'title': source['title'],
        'text': highlighted_text
    })

# 데이터프레임으로 변환하여 표시
df = pd.DataFrame(highlighted_results)
display(df)


,user_id,rating,title,text
0,AFGZNSPIZSAUXINDCQYKB3ISEHQA,5.0,Five Stars,The <em>magazine</em> is the best food <em>mag...
1,AFK4NKFJEKVY73TVAZJXYJCI7Z3Q,4.0,Four Stars,Great <em>magazine</em>. Better than our own c...
2,AFUT7BWYC2YSGJIFIVYLXZIDL4BQ,1.0,One Star,I received one <em>magazine</em> in 1 <em>maga...
3,AGINWBKNR5XQ7VEVBLJSLFETSGSQ,4.0,Out of order,My husband enjoys reading this <em>magazine</em>.
4,AFDUFRWNR4IZXQYCNKWWJRVEV7QQ,5.0,Five Stars,Great <em>magazine</em>. Great articles and in...
5,AFVMGNIF7TZ4YJQP6NM5BGAZ2OOQ,5.0,Five Stars,"Enjoy looking at this <em>magazine</em>, just ..."
6,AHBJNTRUIVIALCTVGDRYFZWM4ZPA,5.0,Not just for husbands 😂,My husband loves this <em>magazine</em>! I lov...
7,AHM22M2U2KULXWOMXICSZQ2WJGYQ,5.0,have always loved this,"Tried and true <em>magazine</em>,have always l..."
8,AEJUXQV3Z2I6LWLHV7GJIS62NP5A,5.0,Five Stars,Great Neue Post <em>Magazine</em>. The best Ge...
9,AFQCM66ZCURMOXYXCRCZIX2NZ5CA,3.0,more decor than structure,more of a decor <em>magazine</em> than an arch...


In [25]:
# 하이라이트된 텍스트 HTML 형식으로 표시
highlighted_html = []

# CSS 스타일 정의
style = """
<style>
em {
    background-color: yellow;
}
</style>
"""

for result in highlighted_results:
    highlighted_html.append(f"""
        <div>
            <h4>{result['title']}</h4>
            <p><strong>User ID:</strong> {result['user_id']}</p>
            <p><strong>Rating:</strong> {result['rating']}</p>
            <p>{result['text']}</p>
        </div>
        <hr>
    """)

# HTML 형식으로 표시
display(HTML(style + ''.join(highlighted_html)))

### 8. 정렬 (Sort)

In [26]:
# 평점에 따라 내림차순 정렬
query = {
    "query": {
        "match_all": {}
    },
    "sort": [
        {"rating": {"order": "desc"}}
    ]
}

all_hits = fetch_all_hits_with_scroll(es, "amazon_reviews", query)
df_hits = pd.DataFrame(all_hits)
print(df_hits.shape)
df_hits.head()

(71497, 6)


,_index,_type,_id,_score,_source,sort
0,amazon_reviews,_doc,8LaX3JABKR8jdWkMoL29,None,"{'rating': 5.0, 'title': 'Great pictures and r...",[5.0]
1,amazon_reviews,_doc,8raX3JABKR8jdWkMoL29,None,"{'rating': 5.0, 'title': 'A great look at what...",[5.0]
2,amazon_reviews,_doc,87aX3JABKR8jdWkMoL29,None,"{'rating': 5.0, 'title': 'If you enjoy reading...",[5.0]
3,amazon_reviews,_doc,9raX3JABKR8jdWkMoL29,None,"{'rating': 5.0, 'title': 'Loving it!', 'text':...",[5.0]
4,amazon_reviews,_doc,97aX3JABKR8jdWkMoL29,None,"{'rating': 5.0, 'title': 'Giftfor mom. She lov...",[5.0]


### Nori 토그나이저

1. elasticsearch container 에서 bash 실행  
```docker exec -it docker-elk-elasticsearch-1 /bin/bash```

2. elasticsearch container 에서 Nori 설치  
```bin/elasticsearch-plugin install analysis-nori``` 

3. Docker에서 컨테이너 재실행

In [47]:
es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': 'elasticsearch', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BdPu_csRRH6ooZBADMrtfQ', 'version': {'number': '7.17.20', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'b26557f585b7d95c71a5549e571a6bcd2667697d', 'build_date': '2024-04-08T08:34:31.070382898Z', 'build_snapshot': False, 'lucene_version': '8.11.3', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [32]:
df_insurance = pd.read_csv("손해보험주요영업활동조회.csv")
df_insurance = df_insurance.fillna('')
df_insurance.head()

,basYm,crno,fncoCd,fncoNm,ClsfAmt,RatDcd,RatDcdNm
0,201912,1101110013328.0,0010626,메리츠화재해상보험주식회사,88.48,A,경과손해율_자동차
1,201709,1101110013328.0,0010626,메리츠화재해상보험주식회사,76.97,A,경과손해율_자동차
2,202109,1101110013328.0,0010626,메리츠화재해상보험주식회사,75.77,A,경과손해율_자동차
3,201609,1101110013328.0,0010626,메리츠화재해상보험주식회사,83.09,A,경과손해율_자동차
4,201403,1101110013328.0,0010626,메리츠화재해상보험주식회사,91.30,A,경과손해율_자동차


In [37]:
es.indices.create(
    index='insurance',
    body={
        
        "mappings": {
            "properties": {
                "crno": {"type": "long"},
                "fncoCd": {"type": "keyword"},
                "fncoNm": {"type": "text"},
                "ClsfAmt": {"type": "float"},
                "RatDcd": {"type": "keyword"},
                "RatDcdNm": {"type": "keyword"},
                "basYm": {
                    "type": "date",
                    "format": "yyyyMM"
                },                    
            }
        },
    },
    ignore=400,
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'insurance'})

In [38]:
action_list = []
doc_id = 1

for row in tqdm(df_insurance.iterrows()):

    doc = {
        "crno": row[1]["crno"],            
        "fncoCd": row[1]["fncoCd"], 
        "fncoNm": row[1]["fncoNm"], 
        "ClsfAmt": row[1]["ClsfAmt"], 
        "RatDcd": row[1]["RatDcd"], 
        "RatDcdNm": row[1]["RatDcdNm"], 
        "basYm": row[1]["basYm"], 
    }
        
        
    record ={
        '_index': 'insurance',
        '_source': doc,
         "_id": int(doc_id)
    }

    action_list.append(record)
    doc_id += 1         

bulk(es, action_list, 
     raise_on_error=False,
    )

20200it [00:01, 15461.11it/s]


(20200, [])

In [51]:
es.indices.create(
    index='insurance-nori',
    body={
        "settings": {
             "analysis": {
                 "analyzer": {
                     "korean": {
                         "tokenizer": "nori_tokenizer"
                     }
                 }
             }
            },
        
        "mappings": {
            "properties": {
                "crno": {"type": "long"},
                "fncoCd": {"type": "keyword"},
                "fncoNm": {"type": "text",  "analyzer":"korean"},
                "ClsfAmt": {"type": "float"},
                "RatDcd": {"type": "keyword"},
                "RatDcdNm": {"type": "keyword"},
                "basYm": {
                    "type": "date",
                    "format": "yyyyMM"
                },                    
            }
        },
    },
    ignore=400,
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'insurance-nori'})

In [52]:
action_list = []
doc_id = 1

for row in tqdm(df_insurance.iterrows()):

    doc = {
        "crno": row[1]["crno"],            
        "fncoCd": row[1]["fncoCd"], 
        "fncoNm": row[1]["fncoNm"], 
        "ClsfAmt": row[1]["ClsfAmt"], 
        "RatDcd": row[1]["RatDcd"], 
        "RatDcdNm": row[1]["RatDcdNm"], 
        "basYm": row[1]["basYm"], 
    }
        
        
    record ={
        '_index': 'insurance-nori',
        '_source': doc,
         "_id": int(doc_id)
    }

    action_list.append(record)
    doc_id += 1         

bulk(es, action_list, 
     raise_on_error=False,
    )

20200it [00:01, 15217.99it/s]


(20200, [])

In [41]:
resp = es.search(index="insurance", 
                 
                 query={
                     "match": {
                     "RatDcd": "C",                       
                    }},                  
                 
                )
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])

Got 1683 Hits:
{'crno': 1101810029112.0, 'fncoCd': '0010645', 'fncoNm': '페더럴인슈런스컴퍼니한국지점', 'ClsfAmt': 0.0, 'RatDcd': 'C', 'RatDcdNm': '경과손해율_장기', 'basYm': 200903}
{'crno': 1101810031646.0, 'fncoCd': '0010649', 'fncoNm': '제너럴재보험주식회사 서울지점', 'ClsfAmt': 0.0, 'RatDcd': 'C', 'RatDcdNm': '경과손해율_장기', 'basYm': 200903}
{'crno': 1101810031894.0, 'fncoCd': '0010650', 'fncoNm': '퍼스트어메리칸권원보험(주)한국지점', 'ClsfAmt': 0.0, 'RatDcd': 'C', 'RatDcdNm': '경과손해율_장기', 'basYm': 200903}
{'crno': 1101110013328.0, 'fncoCd': '0010626', 'fncoNm': '메리츠화재해상보험주식회사', 'ClsfAmt': 75.17, 'RatDcd': 'C', 'RatDcdNm': '경과손해율_장기', 'basYm': 202206}
{'crno': 1101110013328.0, 'fncoCd': '0010626', 'fncoNm': '메리츠화재해상보험주식회사', 'ClsfAmt': 75.84, 'RatDcd': 'C', 'RatDcdNm': '경과손해율_장기', 'basYm': 202203}
{'crno': 1101110006456.0, 'fncoCd': '0010627', 'fncoNm': '한화손해보험주식회사', 'ClsfAmt': 84.12, 'RatDcd': 'C', 'RatDcdNm': '경과손해율_장기', 'basYm': 202206}
{'crno': 1101110006456.0, 'fncoCd': '0010627', 'fncoNm': '한화손해보험주식회사', 'ClsfAmt': 85.48, 'RatDcd':

In [42]:
resp = es.search(index="insurance", 
                 
                 query={
                     "match": {
                     "fncoNm": "한화"
                    }}
                )
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit)

Got 0 Hits:


In [53]:
resp = es.search(index="insurance-nori", 
                 
                 query={
                     "match": {
                     "fncoNm": "한화"
                    }}
                )
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit)

Got 588 Hits:
{'_index': 'insurance-nori', '_type': '_doc', '_id': '41', '_score': 4.012193, '_source': {'crno': 1101110006456.0, 'fncoCd': '0010627', 'fncoNm': '한화손해보험주식회사', 'ClsfAmt': 98.04, 'RatDcd': 'A', 'RatDcdNm': '경과손해율_자동차', 'basYm': 201912}}
{'_index': 'insurance-nori', '_type': '_doc', '_id': '42', '_score': 4.012193, '_source': {'crno': 1101110006456.0, 'fncoCd': '0010627', 'fncoNm': '한화손해보험주식회사', 'ClsfAmt': 79.68, 'RatDcd': 'A', 'RatDcdNm': '경과손해율_자동차', 'basYm': 201709}}
{'_index': 'insurance-nori', '_type': '_doc', '_id': '43', '_score': 4.012193, '_source': {'crno': 1101110006456.0, 'fncoCd': '0010627', 'fncoNm': '한화손해보험주식회사', 'ClsfAmt': 80.74, 'RatDcd': 'A', 'RatDcdNm': '경과손해율_자동차', 'basYm': 202109}}
{'_index': 'insurance-nori', '_type': '_doc', '_id': '44', '_score': 4.012193, '_source': {'crno': 1101110006456.0, 'fncoCd': '0010627', 'fncoNm': '한화손해보험주식회사', 'ClsfAmt': 83.72, 'RatDcd': 'A', 'RatDcdNm': '경과손해율_자동차', 'basYm': 201609}}
{'_index': 'insurance-nori', '_type': 

### [실습] 
loan_payments_data.csv 데이터를 Elasticserach에 Indexing하고 다양한 쿼리를 연습합니다.